In [1]:
import numpy as np
import pandas as pd
import catboost as cbt
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

train = pd.read_csv('../data/first_round_training_data.csv')
test = pd.read_csv('../data_3/second_round_testing_data.csv')
submit = pd.read_csv('../data_3/submit_example2.csv')
data = train.append(test).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)


##取关于属性的平均值
tmp = data.groupby('Parameter8',as_index=False)['Attribute6'].mean()
del data['Attribute6']
data= data.merge(tmp,how='left')

tmp = data.groupby('Parameter8',as_index=False)['Attribute4'].mean()
del data['Attribute4']
data= data.merge(tmp,how='left')

tmp = data.groupby('Parameter8',as_index=False)['Attribute9'].mean()
del data['Attribute9']
data= data.merge(tmp,how='left')



data['ch910'] = np.log10(data['Parameter9']/data['Parameter10'])
data['ch56_add'] = data['Parameter5']+data['Parameter6']  ##u
data['ch56_mul'] = data['Parameter5']*data['Parameter6']
data['cnt910'] = data.groupby('Parameter9')['Parameter10'].transform('count')
data['cnt78'] = data.groupby('Parameter7')['Parameter8'].transform('count')


feature_name = ['Parameter{0}'.format(i) for i in range(5, 11)] +['ch56_add','ch56_mul','ch910','cnt910','Attribute6','Attribute4','Attribute9']
tr_index = ~data['label'].isnull()
X_train = data[tr_index].reset_index(drop=True)
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)

In [2]:
feature_name.remove('Attribute6')
feature_name.remove('Attribute4')
tmp_pre = np.zeros((len(X_test),4))
print(feature_name)
sub1 = pd.DataFrame()
for i in range(5):
    iterations = 1301
    cbt_model = cbt.CatBoostClassifier(iterations=iterations,learning_rate=0.05,verbose=300,
    early_stopping_rounds=1000,task_type='GPU',max_depth=7,
    loss_function='MultiClass',random_state=i*2244+50)
    cbt_model.fit(X_train[feature_name], y ,eval_set=(X_train[feature_name],y))
    oof = cbt_model.predict_proba(X_train[feature_name])
    prediction = cbt_model.predict_proba(X_test[feature_name])
    print(np.corrcoef(prediction[:,0],tmp_pre[:,0] ) )  ##比较不同种子下相似度 0.991 lgb模型相似度可能高些
    
    tmp_pre = cbt_model.predict_proba(X_test[feature_name])
    
    print(cbt_model.feature_importances_)

    sub = test[['Group']]
    prob_cols = [i for i in submit.columns if i not in ['Group']]
    for i, f in enumerate(prob_cols):
        sub[f] = prediction[:, i]
    sub1 = sub1.append(sub)    
    


['Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10', 'ch56_add', 'ch56_mul', 'ch910', 'cnt910', 'Attribute9']
0:	learn: -1.3610142	test: -1.3610145	best: -1.3610145 (0)	total: 7.05ms	remaining: 9.17s
300:	learn: -0.9748193	test: -0.9748193	best: -0.9748193 (300)	total: 1.51s	remaining: 5.02s
600:	learn: -0.9086890	test: -0.9086891	best: -0.9086891 (600)	total: 2.94s	remaining: 3.42s
900:	learn: -0.8626077	test: -0.8626077	best: -0.8626077 (900)	total: 4.35s	remaining: 1.93s
1200:	learn: -0.8270229	test: -0.8270230	best: -0.8270230 (1200)	total: 5.5s	remaining: 458ms
1300:	learn: -0.8165463	test: -0.8165463	best: -0.8165463 (1300)	total: 5.91s	remaining: 0us
bestTest = -0.8165463053
bestIteration = 1300
Shrink model to first 1301 iterations.
[[ 1. nan]
 [nan nan]]
[11.46551392  8.626522    5.60946013  5.63205127  2.49725347 26.50644416
  6.8699454   6.87255627 18.27939323  3.50650667  4.1343535 ]
0:	learn: -1.3606323	test: -1.3606322	best: -1.3606322 (0)

In [3]:
sub1.shape

(30000, 5)

In [4]:
for i, f in enumerate(prob_cols):
    sub1[f+'_rank'] = sub1.groupby('Group')[f].rank(method='first')
    sub1[f+'_rank'] = sub1[f+'_rank']-125
    sub1[f+'_rank'] = sub1[f+'_rank'].apply(lambda x: -(0.8/125/125/250)*x*x+1.4/250 )
    sub1[f+'_rank'] = sub1[f+'_rank']  #*sub['gcnt']
    sub1[f] = sub1[f]*sub1[f+'_rank']
for i in prob_cols:
    sub1[i] = sub1.groupby('Group')[i].transform('sum')  

# sub1 = sub1[list(submit)].drop_duplicates()
# sub1['all_sum'] = sub1[prob_cols].sum(axis=1)

In [5]:
sub1 = sub1[list(submit)].drop_duplicates().reset_index(drop = True)
sub1['all_sum'] = sub1[prob_cols].sum(axis=1)

In [6]:
sub1.shape

(120, 6)

In [7]:

for i in prob_cols:
    sub1[i] = sub1[i]/sub1['all_sum']
    
# sub1['Group'] = submit['Group']


In [12]:
sub1[list(submit)].to_csv('../EDA/result_final/cat.csv',index=False) ###0.70104530000 


In [13]:
sub1

,Group,Excellent ratio,Good ratio,Pass ratio,Fail ratio,all_sum
0,0,0.217391,0.334086,0.256876,0.191647,1.101563
1,1,0.173625,0.264973,0.393564,0.167837,1.079232
2,2,0.163378,0.355196,0.345895,0.135531,1.104639
3,3,0.241559,0.312700,0.344231,0.101510,1.090593
4,4,0.168310,0.412493,0.269224,0.149974,1.100926
5,5,0.184341,0.249479,0.408708,0.157472,1.074924
6,6,0.212556,0.332741,0.331267,0.123436,1.102559
7,7,0.158052,0.252365,0.441595,0.147987,1.097144
8,8,0.134468,0.151884,0.637514,0.076134,1.111446
9,9,0.139646,0.283832,0.382621,0.193901,1.070562


In [16]:
submit['Group']

0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
10      10
11      11
12      12
13      13
14      14
15      15
16      16
17      17
18      18
19      19
20      20
21      21
22      22
23      23
24      24
25      25
26      26
27      27
28      28
29      29
30      30
31      31
32      32
33      33
34      34
35      35
36      36
37      37
38      38
39      39
40      40
41      41
42      42
43      43
44      44
45      45
46      46
47      47
48      48
49      49
50      50
51      51
52      52
53      53
54      54
55      55
56      56
57      57
58      58
59      59
60      60
61      61
62      62
63      63
64      64
65      65
66      66
67      67
68      68
69      69
70      70
71      71
72      72
73      73
74      74
75      75
76      76
77      77
78      78
79      79
80      80
81      81
82      82
83      83
84      84
85      85
86      86
87      87
88      88
89      89
90      90